In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
import zipfile

!unzip /content/drive/MyDrive/data/train.csv.zip

In [ ]:
import glob

File_paths = glob.glob("/content/drive/MyDrive/data/train.csv")

In [ ]:
train_folder = r"/content/drive/MyDrive/data/train.csv"

In [ ]:
train_folder

In [ ]:
import os
os.path.getsize("/content/train.csv")

### Read Data With Pandas

In [ ]:
import pandas as pd
import time
start = time.time()
data_pd=pd.read_csv("/content/train.csv", nrows=10000000)
end = time.time()
print()
print("Time taken to read with pandas: ",(end-start),"seconds")
print()

### Read Data With Dask

In [ ]:
from dask import dataframe as dd
start = time.time()
data_dd=pd.read_csv("/content/train.csv", nrows=10000000)
end = time.time()
print()
print("Time taken to read with Dask: ",(end-start),"seconds")
print()

### Read Data With Modin

In [ ]:
pip install modin


In [ ]:
import modin.pandas as mpd
start = time.time()
data_mpd=pd.read_csv("/content/train.csv", nrows=10000000)
end = time.time()
print()
print("Time taken to read with Modin: ",(end-start),"seconds")
print()

In [ ]:
# dask is better than other two i.e., modin and pandas
# 2.24 sec lesser than pandas
# 0.09 sec lesser than modin

In [ ]:
data_dd.head()

In [ ]:
data_dd.info()

In [ ]:
data_dd.columns

In [ ]:
df=data_dd.copy()

In [ ]:

import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

In [ ]:
df.info()

In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: file
file_name: train
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
      - row_id
      - timestamp
      - user_id
      - content_id
      - content_type_id
      - task_container_id
      - user_answer
      - answered_correctly
      - prior_question_elapsed_time
      - prior_question_had_explanation

In [ ]:
import yaml
import testutility as util
config_data = util.read_config_file("/content/file.yaml")

In [ ]:
config_data['inbound_delimiter']

In [ ]:
#inspecting data of config file
config_data

In [ ]:
#Reading the file using config file
file_type = config_data['file_type']
source_file ="./" + config_data['file_name'] + f'.{file_type}'

In [ ]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'],nrows=10000000)
df.head()

In [ ]:
#validating the header of the file
util.col_header_val(df,config_data)

In [ ]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")